# Natural Vibration of a Bernoulli Console

In [1]:
import texttable
import numpy as np


def print_frequencies(m_eff, freq, tol=10):
    tableObj = texttable.Texttable()
    rows = [['', "mx", "my", "mz", "freq"]]
    for i in range(freq.shape[0]):
        if np.max(m_eff[i]) > tol:
            rows.append([i+1, m_eff[i, 0], m_eff[i, 1], m_eff[i, 2], freq[i]])
    tableObj.add_rows(rows)
    print(tableObj.draw())
    

A short comparison of different modeling techniques of a simple console. The beam has a rectangular prismatic cross-section and a linear elastic material model, governed by the following parameters:

In [2]:
# all units in kN and cm
L = 150.  # length of the console [cm]
F = 1.  # value of the vertical load at the free end [kN]
E = 21000.0  # Young's modulus [kN/cm3]
nu = 0.3  # Poisson's ratio [-]
w, h = 5., 15.  # width and height of the rectangular cross section
g = 9.81  # gravitational acceleration [m/s2]
density = 7750.0 * 1e-6 # mass density [g/cm3]
nElem = 20  # number of subdivisons to use

In [3]:
A = w * h  # area
Iy = w * h**3 / 12  # second moment of inertia around the y axis
Iz = h * w**3 / 12  # second moment of inertia around the z axis
Ix = Iy + Iz  # torsional inertia
weight = density * 1e-3 * g  # [kN/cm3]
dpa = density * A  # density per area [g/cm]
wpa = weight * A  # weight per area [kN/cm]
mass = L * dpa  # total mass

In [4]:
from sigmaepsilon import Structure, LineMesh, PointData
from neumann.linalg import linspace, Vector
from neumann import repeat
from polymesh.space import StandardFrame, PointCloud
from sigmaepsilon.fem.cells import B2 as Beam
import numpy as np

# model stiffness matrix
G = E / (2 * (1 + nu))
Hooke = np.array([
    [E*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, E*Iy, 0],
    [0, 0, 0, E*Iz]
])

# space
GlobalFrame = StandardFrame(dim=3)

# mesh
p0 = np.array([0., 0., 0.])
p1 = np.array([L, 0., 0.])
coords = linspace(p0, p1, nElem+1)
coords = PointCloud(coords, frame=GlobalFrame).show()
topo = np.zeros((nElem, 2), dtype=int)
topo[:, 0] = np.arange(nElem)
topo[:, 1] = np.arange(nElem) + 1

# load at the rightmost node in Y and Z directions
nodal_loads = np.zeros((coords.shape[0], 6, 3))
global_load_vector = Vector([F,  0, 0.], frame=GlobalFrame).show()
nodal_loads[-1, :3, 0] = global_load_vector
global_load_vector = Vector([0., F, 0.], frame=GlobalFrame).show()
nodal_loads[-1, :3, 1] = global_load_vector
global_load_vector = Vector([0., 0, F], frame=GlobalFrame).show()
nodal_loads[-1, :3, 2] = global_load_vector

# support at the leftmost node (all degrees)
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
fixity[0, :] = True
fixity=fixity.astype(float) * 1e40

# mass and density
nodal_masses = np.zeros((coords.shape[0],))
densities = np.full((topo.shape[0],), dpa)

# pointdata
pd = PointData(coords=coords, loads=nodal_loads, 
               fixity=fixity, mass=nodal_masses)

# celldata
#frames = frames_of_lines(coords, topo)
frames = repeat(GlobalFrame.show(), topo.shape[0])
cd = Beam(topo=topo, frames=frames, material=Hooke, density=densities)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)

# perform linear analysis
structure.linear_static_analysis()

# postproc
dofsol = structure.nodal_dof_solution()

## Dunkerley's Approximation

Lower bound estimations on the smallest natural circular frequencies by *splitting the masses*.

In [5]:
def Dunkerley(N:int=1):
    mi = mass / N
    h = L / N
    i1 = np.sum(list(range(1, N + 1)))
    i3 = np.sum([i**3 for i in range(1, N + 1)])
    acc_x = i1 * mi * h / (E * A)
    acc_y = i3 * mi * h**3 / (3 * E * Iz)
    acc_z = i3 * mi * h**3 / (3 * E * Iy)
    return np.sqrt(1/acc_x), np.sqrt(1/acc_y), np.sqrt(1/acc_z)


Dunkerley(20)

(15.14564891325506, 0.3483830229642758, 1.0451490688928273)

## Rayleigh's Approximtion

Upper bound estimations of the smallest natural circular frequencies using the equivalence of elastic internal energy and kinetic energy of undamped conservative systems.

In [6]:
from sigmaepsilon.fem.dyn import Rayleigh_quotient


def Rayleigh(structure: Structure):
    M = structure.mass_matrix(penalize=True)
    u = structure.nodal_dof_solution(flatten=True)
    f = structure.nodal_forces(flatten=True)
    return np.sqrt(Rayleigh_quotient(M, u=u, f=f))


Rayleigh(structure)

array([19.00763853,  0.37668143,  1.13004428])

## FEM Approximation

By solving the eigenvalue problem using dense matrices:

In [7]:
structure.cleanup()
structure.free_vibration_analysis(normalize=True, as_dense=True)
freqs = structure.natural_circular_frequencies()
freqs[:20]

array([3.16156119e-03, 3.16211113e-03, 3.16225821e-03, 3.16226955e-03,
       3.16227766e-03, 3.16243056e-03, 3.71245829e-01, 1.11373749e+00,
       2.32512847e+00, 6.50404542e+00, 6.97538540e+00, 1.27274400e+01,
       1.72424522e+01, 1.95121363e+01, 2.10021166e+01, 3.13076967e+01,
       3.81823201e+01, 4.36234097e+01, 4.88080816e+01, 5.18337700e+01])

In [8]:
around = min(Rayleigh(structure))
freks, modes = structure.natural_circular_frequencies(k=40, return_vectors=True, 
                                                      which='SM')
freks[np.where(np.abs(freks) < 1e-3)] = 0.
freks[:20]

array([3.16156119e-03, 3.16211113e-03, 3.16225821e-03, 3.16226955e-03,
       3.16227766e-03, 3.16243056e-03, 3.71245829e-01, 1.11373749e+00,
       2.32512847e+00, 6.50404542e+00, 6.97538540e+00, 1.27274400e+01,
       1.72424522e+01, 1.95121363e+01, 2.10021166e+01, 3.13076967e+01,
       3.81823201e+01, 4.36234097e+01, 4.88080816e+01, 5.18337700e+01])

Using Rayleigh's approximation to help the sparse solver:

In [9]:
around = min(Rayleigh(structure))**2
freks, modes = structure.natural_circular_frequencies(k=40, around=around, 
                                                      return_vectors=True, which='SM')
freks[np.where(np.abs(freks) < 1e-3)] = 0.
freks[:20]

array([3.16156119e-03, 3.16211113e-03, 3.16225821e-03, 3.16226955e-03,
       3.16227766e-03, 3.16243056e-03, 3.71245829e-01, 1.11373749e+00,
       2.32512847e+00, 6.50404542e+00, 6.97538540e+00, 1.27274400e+01,
       1.72424522e+01, 1.95121363e+01, 2.10021166e+01, 3.13076967e+01,
       3.81823201e+01, 4.36234097e+01, 4.88080816e+01, 5.18337700e+01])

## Effective Modal Masses

In [10]:
from sigmaepsilon.fem.dyn import effective_modal_masses

nN, nD = len(coords), 6

action_x = np.zeros((nN, nD))
action_x[:, 0] = 1.0
action_x = action_x.reshape(nN * nD)
action_y = np.zeros((nN, nD))
action_y[:, 1] = 1.0
action_y = action_y.reshape(nN * nD)
action_z = np.zeros((nN, nD))
action_z[:, 2] = 1.0
action_z = action_z.reshape(nN * nD)
actions = np.stack([action_x, action_y, action_z], axis=1)

M = structure.mass_matrix(penalize=False)
m_eff = effective_modal_masses(M, actions, modes)

print_frequencies(m_eff, freks, 30)

+----+--------+--------+--------+--------+
|    |   mx   |   my   |   mz   |  freq  |
+====+========+========+========+========+
| 7  | 0.000  | 53.441 | 0.000  | 0.371  |
+----+--------+--------+--------+--------+
| 8  | 0.000  | 0.000  | 53.441 | 1.114  |
+----+--------+--------+--------+--------+
| 13 | 70.671 | 0.000  | 0.000  | 17.242 |
+----+--------+--------+--------+--------+


The sum of effective masses for an action converges to the total mass of the structure as the mesh is refined, but since some masses get distributed to fixed supports, it will never reach the total mass. 

In [11]:
mass_fem = [np.sum(m_eff[:, i]) for i in range(3)]
mass, mass_fem

(87.1875, [85.92905683512579, 85.70594449617565, 85.70594449617568])

The percetage of the sums of effective masses to the total mass:

In [12]:
["{:.2f}%".format(100 * m / mass) for m in mass_fem]

['98.56%', '98.30%', '98.30%']

Find eigenvalues with largest effective masses:

In [13]:
imax = [np.argmax(m_eff[:, i]) for i in range(3)]
freks[imax]

array([17.24245222,  0.37124583,  1.11373749])

Check if the FEM approximation is between lower and upper bound estimations:

In [14]:
f_lower = Dunkerley(20)
f_upper = Rayleigh(structure)
f_fem = freks[imax]
[f_fem[i] > f_lower[i] and f_fem[i] < f_upper[i] for i in range(3)]

[True, True, True]

## Nodal Masses

In the previous FEM approximation we represented the mass by defining density values for the cells. Although we always have to define a density distribution for the cells, this can be augmented by nodal masses. The following block distributes the masses to the nodes, and cell-densities are used to guarantee that the diagonal of the mass matrix is always filled up.

Note that this calculation is inferior to the variational approach, since rotational masses are neglected, as all the mass is delegated to translational DOFs.

In [15]:
# mass and density
nN = coords.shape[0]  # the number of nodes in the model
min_cell_mass = mass / 1000
cell_density = min_cell_mass / L
densities = np.full((topo.shape[0],), cell_density)
mass_on_nodes = (mass - min_cell_mass) / nN
nodal_masses = np.full((coords.shape[0],), mass_on_nodes)

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame, loads=nodal_loads, 
               fixity=fixity, mass=nodal_masses)

# celldata
frames = repeat(GlobalFrame.show(), topo.shape[0])
cd = Beam(topo=topo, frames=frames, material=Hooke, density=densities)

# set up mesh and structure
mesh = LineMesh(pd, cd, frame=GlobalFrame)
structure = Structure(mesh=mesh)
structure.linear_static_analysis()

# postproc
dofsol = structure.nodal_dof_solution()

In [16]:
structure.free_vibration_analysis(normalize=True, as_dense=True)
freks, modes = structure.natural_circular_frequencies(return_vectors=True)
M = structure.mass_matrix(penalize=False)
u = structure.nodal_dof_solution(flatten=True)
f = structure.nodal_forces(flatten=True)

nN, nD, nR = dofsol.shape

action_x = np.zeros((nN, nD))
action_x[:, 0] = 1.0
action_x = action_x.reshape(nN * nD)
action_y = np.zeros((nN, nD))
action_y[:, 1] = 1.0
action_y = action_y.reshape(nN * nD)
action_z = np.zeros((nN, nD))
action_z[:, 2] = 1.0
action_z = action_z.reshape(nN * nD)
actions = np.stack([action_x, action_y, action_z], axis=1)

m_eff = effective_modal_masses(M, actions, modes)
mtot = [np.sum(m_eff[:, i]) for i in range(3)]

print_frequencies(m_eff, freks, 20)

mass_fem = [np.sum(m_eff[:, i]) for i in range(3)]
["{:.2f}%".format(100 * m / mass) for m in mass_fem]

+----+--------+--------+--------+--------+
|    |   mx   |   my   |   mz   |  freq  |
+====+========+========+========+========+
| 7  | 0.000  | 52.205 | 0.000  | 0.362  |
+----+--------+--------+--------+--------+
| 8  | 0.000  | 0.000  | 52.205 | 1.087  |
+----+--------+--------+--------+--------+
| 13 | 68.923 | 0.000  | 0.000  | 17.229 |
+----+--------+--------+--------+--------+


['95.24%', '95.24%', '95.24%']

Check Rayleigh's approximation with the new mass matrix:

In [17]:
Rayleigh(structure)

array([18.7746361 ,  0.36662324,  1.09986973])

## Plotting with Matplotlib

In [18]:
%matplotlib qt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from latexdocs.utils import floatformatter

f2s = floatformatter(sig=4)

fps = 120
i = 10  # index of the mode of interest

omega = freks[i]
T = 2 * np.pi / omega
u = mesh.pd.vshapes[:, :, i] * 0.1
x = mesh.pd.x[:, 0]
dt = T / fps
f = 1/T

fig, ax = plt.subplots()
title = "f : {f} [1/s]    T : {T} [s]".format(
    f = f2s.format(f), T = f2s.format(T)
)
ax.set_title(title)
line, = ax.plot(x, u[:, 2] * np.sin(omega * 0))


def init():
    line.set_ydata(u[:, 2] * np.sin(omega * 0))
    return line,


def animate(i):
    t = i * dt
    line.set_ydata(u[:, 2] * np.sin(omega * t))  # update the data
    return line,


ani = animation.FuncAnimation(fig, animate, fps, init_func=init,
                              interval=1000*dt, blit=True)

plt.show()